In [ ]:
from chat_with_nerf.chat.agent import Agent 
import os
import json
import numpy as np
from chat_with_nerf.chat.session import Session
import time
import open3d as o3d
from pathlib import Path
from chat_with_nerf.chat.system_prompt import EDITED_SYSTEM_PROMPT, NO_VISUAL_FEEDBACK_SYSTEM_PROMPT
from chat_with_nerf.settings import Settings
from joblib import Parallel, delayed
from evaluation_vis_util import draw_plotly, create_bbox
from tqdm import tqdm
import uuid
from collections import OrderedDict
from utils import is_label_unique, convert_origin_bbox, get_transformation_matrix, construct_bbox_corners, get_box3d_min_max, box3d_iou

In [ ]:
def get_val_set(root_directory):
    json_dict = {}
    # List of all subfolders and their files
    subfolders_files = [(dp, filenames) for dp, _, filenames in os.walk(root_directory)]
    # Dictionary comprehension to pick only the first JSON from each subfolder
    json_dict = {os.path.basename(dp): os.path.join(dp, filenames[0]) for dp, filenames in subfolders_files if any(fn.endswith('.json') for fn in filenames)}

    return json_dict

In [ ]:
root_directory = ''  # Assuming current directory, adjust path if needed

In [ ]:
# GPT35
# os.environ['API_URL'] = ""
# os.environ['OPENAI_API_KEY'] = ""
# GPT 4
# os.environ['API_URL'] = ""
# os.environ['OPENAI_API_KEY'] = ""

In [ ]:
json_dict = get_val_set(root_directory)

In [ ]:
acc_25 = 0
acc_50 = 0
acc_25_unique = 0
acc_50_unique = 0
acc_25_multiple = 0
acc_50_multiple = 0
list_iou = []
total_object = 0
total_unique_object = 0
total_multiple_object = 0
session_id_list = []
acc_25_top2_hit = 0
acc_25_top3_hit = 0
acc_25_top5_hit =0
acc_25_all_hit = 0
acc_25_top2_hit_unique = 0
acc_25_top2_hit_multiple = 0
acc_25_top3_hit_unique = 0
acc_25_top3_hit_multiple = 0
acc_25_top5_hit_unique = 0
acc_25_top5_hit_multiple = 0
acc_25_all_hit_unique = 0
acc_25_all_hit_multiple = 0


result_dict ={
    'scene_name': list(),
    'description': list(),
    'centroid_list': list(),
    'extent_list': list(),
    'similarity_mean_list_list': list(),
    'ground truth': list()
}

In [ ]:
def process_description(agent, scene_name, description, corners_original, is_unique, object_id, query_rank_id, center_original, extents_original):  
    MAX_RETRIES = 3
    for retry in range(MAX_RETRIES):
        try:
            new_session = Session.create_for_scene(scene_name)  
            unique_id = str(uuid.uuid4())
            new_session.session_id = f"{scene_name}-{object_id}-{query_rank_id}-{unique_id}" 
            new_session.working_scene_name = scene_name
            new_session.grounding_query = description
            new_session.ground_truth = [center_original, extents_original]
            print(description)  
            center_list, box_size_list, values_list = agent.act_no_gpt(  
                description,  
                scene_name,
                new_session  
            )  
            combined_list = list(zip(values_list, center_list, box_size_list))
            sorted_list = sorted(combined_list, key=lambda x: x[0], reverse=True)  # reverse=True for descending order
            ordered_result = [(center, box_size) for _, center, box_size in sorted_list]

            top1_hit_list = []
            top2_hit_list = []
            top3_hit_list = []
            top5_hit_list = []
            all_hit_list = []
            
            for index, (center, box_size) in enumerate(ordered_result):
                cur_prediction = construct_bbox_corners(center, box_size)
                iou3d_top = box3d_iou(np.array(corners_original), cur_prediction)
                if index < 1:
                    top1_hit_list.append(iou3d_top)
                if index < 2:
                    top2_hit_list.append(iou3d_top)
                if index < 3:
                    top3_hit_list.append(iou3d_top)
                if index < 5:
                    top5_hit_list.append(iou3d_top)
                all_hit_list.append(iou3d_top)
            
            top1_hit = max(top1_hit_list)
            top2_hit = max(top2_hit_list)
            top3_hit = max(top3_hit_list)
            top5_hit = max(top5_hit_list)
            all_hit = max(all_hit_list)
            
            new_session.center_list = [arr.tolist() for arr in center_list]
            new_session.box_size_list = [list(arr) for arr in box_size_list]
            new_session.values_list = values_list
            
            new_session.save("/workspace/chat-with-nerf-dev/chat-with-nerf/session_output_lerf_14_with_baseline_final")
            return top1_hit, top2_hit, top3_hit, top5_hit, all_hit, new_session.session_id, new_session, is_unique
        except Exception as exp:
            if retry < MAX_RETRIES - 1:  # If it's not the last retry
                print(f"Attempt {retry + 1} failed. Retrying... Error is {exp}")
                continue
            else:  # On the last retry, print the exception and return a message
                print(exp)
                # return f"Failed after {MAX_RETRIES} attempts.", None, None, None

In [ ]:
preprocessed_data = []

In [ ]:
def process_scene(scene_name, json_path, json_dict):
    scene_path = json_dict[scene_name]
    with open(scene_path, 'r') as file:
        data = json.load(file)
    
    furnitures = data['objects']
    agent = Agent(scene_name=scene_name)
    
    results = []
    total_unique_object = 0
    total_multiple_object = 0
    total_object = 0
    preprocessed_data = []

    for furniture in furnitures:
        bbox = furniture['bbox']
        center_original, extents_original = bbox[:3], bbox[3:6]
        corners_original = construct_bbox_corners(center_original, extents_original)
        label = furniture['label']
        is_unique = is_label_unique(furnitures, label)
        
        if is_unique:
            total_unique_object += len(furniture['description'])
        else:
            total_multiple_object += len(furniture['description'])
        
        for idx, description in enumerate(furniture['description']):
            total_object += 1
            preprocessed_data.append((agent, scene_name, description, corners_original, is_unique, label, idx, center_original, extents_original))

    return total_unique_object, total_multiple_object, total_object, preprocessed_data


In [ ]:
# Only include scenes that are within the first 7 entries
selected_scenes = list(json_dict.items())[:7]

results = Parallel(n_jobs=7)(delayed(process_scene)(scene_name, json_path, json_dict) for scene_name, json_path in selected_scenes)

# Collate results
total_unique_object = sum(result[0] for result in results)
total_multiple_object = sum(result[1] for result in results)
total_object = sum(result[2] for result in results)
preprocessed_data = [item for result in results for item in result[3]]

In [ ]:
results = Parallel(n_jobs=5, backend='threading')(delayed(process_description)(*data) for data in tqdm(preprocessed_data, desc="Processing descriptions"))

In [ ]:
list_iou = []

In [ ]:
for result in results:
    if result is None:
        continue

    top1_hit, top2_hit, top3_hit, top5_hit, all_hit, session_id, session, is_unique = result
    
    if top1_hit:
        if top1_hit > 0.25:  
            acc_25 += 1
            if is_unique:  
                acc_25_unique += 1  
            else:
                acc_25_multiple += 1  
        if top1_hit > 0.5:
            acc_50 += 1  
            if is_unique:  
                acc_50_unique += 1  
            else:  
                acc_50_multiple += 1
    else:
        top1_hit = 0
        sessionid = 'none'
    
    if top2_hit:
        if top2_hit > 0.25:
            acc_25_top2_hit += 1
            if is_unique:
                acc_25_top2_hit_unique += 1
            else:
                acc_25_top2_hit_multiple += 1
    
    if top3_hit:
        if top3_hit > 0.25:
            acc_25_top3_hit += 1
            if is_unique:
                acc_25_top3_hit_unique += 1
            else:
                acc_25_top3_hit_multiple += 1
    
    if top5_hit:
        if top5_hit > 0.25:
            acc_25_top5_hit += 1
            if is_unique:
                acc_25_top5_hit_unique += 1
            else:
                acc_25_top5_hit_multiple += 1
                
    if all_hit:
        if all_hit > 0.25:
            acc_25_all_hit += 1
            if is_unique:
                acc_25_all_hit_unique += 1
            else:
                acc_25_all_hit_multiple += 1
                
                
    list_iou.append(top1_hit)
    session_id_list.append(session_id)

In [ ]:
print("acc_25 =", acc_25)
print("acc_50 =", acc_50)
print("acc_25_unique =", acc_25_unique)
print("acc_50_unique =", acc_50_unique)
print("acc_25_multiple =", acc_25_multiple)
print("acc_50_multiple =", acc_50_multiple)
print("acc_25_top2_hit =", acc_25_top2_hit)
print("acc_25_top3_hit =", acc_25_top3_hit)
print("acc_25_top5_hit =", acc_25_top5_hit)
print("acc_25_all_hit =", acc_25_all_hit)
print('total_object: ', total_object)
print('total_unique_object: ', total_unique_object)
print('total_multiple_object: ', total_multiple_object)
print("acc_25_top2_hit_unique: ", acc_25_top2_hit_unique)
print("acc_25_top2_hit_multiple: ", acc_25_top2_hit_multiple)
print("acc_25_top3_hit_unique: ", acc_25_top3_hit_unique)
print("acc_25_top3_hit_multiple: ", acc_25_top3_hit_multiple)
print("acc_25_top5_hit_unique: ", acc_25_top5_hit_unique)   
print("acc_25_top5_hit_multiple: ", acc_25_top5_hit_multiple)
print("acc_25_all_hit_unique: ", acc_25_all_hit_unique)
print("acc_25_all_hit_multiple: ", acc_25_all_hit_multiple)
print("list_iou =", list_iou)
print("total_object =", total_object)
print("session_id_list =", session_id_list)

In [ ]:
top1_hit, top2_hit, top3_hit, top5_hit, all_hit, session_id, session, is_unique = results[0]